In [2]:
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import messagebox
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the dataset
data = pd.read_csv('GOOGL.csv')

# Extract 'Close' prices
prices = data['Close'].values.reshape(-1, 1)

# Normalize the prices
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_prices = scaler.fit_transform(prices)

# Function to create sequences for RNN
def create_sequences(data, seq_length):
  X, y = [], []
  for i in range(len(data) - seq_length):
    X.append(data[i:i + seq_length])
    y.append(data[i + seq_length])
  return np.array(X), np.array(y)

# Create sequences with a sequence length of 30 days
sequence_length = 30
X, y = create_sequences(normalized_prices, sequence_length)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Build the RNN model
model = Sequential([
  LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)),
  LSTM(units=50),
  Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# Predictions on test set
y_pred = model.predict(X_test)

# Denormalize predictions and actual prices
y_pred_denormalized = scaler.inverse_transform(y_pred)
y_test_denormalized = scaler.inverse_transform(y_test)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test_denormalized, y_pred_denormalized)


# Function to visualize actual vs predicted prices
def visualize_predictions():
  plt.plot(y_test_denormalized, label='Actual Price')
  plt.plot(y_pred_denormalized, label='Predicted Price')
  plt.xlabel('Time Step')
  plt.ylabel('Price')
  plt.legend()
  plt.title('Actual vs. Predicted Prices')
  plt.show()

# GUI
def predict_price():
  try:
    # Get user input
    lookback_days = int(entry.get())

    # Prepare input data
    last_sequence = normalized_prices[-sequence_length:]
    X_input = last_sequence.reshape(1, sequence_length, 1)

    # Predict future prices
    predicted_price = model.predict(X_input)

    # Denormalize predicted price
    predicted_price = scaler.inverse_transform(predicted_price)

    # Display the predicted price and accuracy (MAE)
    result_label.config(text=f"Predicted Price: ${predicted_price[0][0]:.2f}\nMAE on Test Data: {mae:.2f}")

  except Exception as e:
    messagebox.showerror("Error", str(e))

# Create tkinter window
window = tk.Tk()
window.title("Stock Price Prediction")

# Create GUI elements
label = tk.Label(window, text="Enter number of past days to consider:")
label.pack()

entry = tk.Entry(window)
entry.pack()

button_predict = tk.Button(window, text="Predict", command=predict_price)
button_predict.pack()

button_visualize = tk.Button(window, text="Visualize", command=visualize_predictions)
button_visualize.pack()

result_label = tk.Label(window, text="")
result_label.pack()

# Run the GUI main loop
window.mainloop()


C:\Users\rutuj\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.0024
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 4.0164e-05
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 4.2492e-05
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 3.4591e-05
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 4.0911e-05
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 4.1886e-05
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 3.7006e-05
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 3.7397e-05
Epoch 9/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4.0681e-05
Epoch 10/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 3.2886e-05
Epoch 11/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 3.6236e-05
Epoch 12/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 2.7055e-05
Epoch 13/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 2.6861e-05
Epoch 14/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 3.